In [206]:
import pandas as pd
import numpy as np
import sys
import os
import comtypes.client
import matplotlib.pyplot as plt

# <center>**Find and Attach to ETABS Instance**<center>

In [207]:
# Note: can only attach to one active ETABS model. Please close all other models.
helper = None

# Create API helper object.
helper = comtypes.client.CreateObject('ETABSv1.Helper')
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)  # type: ignore

try:
    # Get the current ETABS instance.
    myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject")
except (OSError, comtypes.COMError):
    print("No running instance of the program found or failed to attach.")
    sys.exit(-1)

# Create SapModel object (ETABS instance)
try:
    SapModel = myETABSObject.SapModel
    print("Successfully connected to ETABS instance.")
except AttributeError:
    print("No running instance of the program found or failed to attach.")

SapModel = None  # Resetting to None before reinitializing
SapModel = myETABSObject.SapModel

Successfully connected to ETABS instance.


# <center>**Extract Story Height and Elevations**<center>

In [208]:
# Grab the story definitions table.
story_definition_table = SapModel.DatabaseTables.GetTableForDisplayArray("Story Definitions", GroupName = "") # type: ignore

# Extract the column headings and number of rows from the table. This needs to be done as the data is currently an array.
cols = story_definition_table[2]
num_rows = story_definition_table[3]

# Input data into array split.
table_data = np.array_split(story_definition_table[4], num_rows)

# Create story definitions dataframe.
storyDefinitions_df = pd.DataFrame(table_data)
storyDefinitions_df.columns = cols

# Delete all columns except story and height.
storyDefinitions_df = storyDefinitions_df.drop(['Tower', 'IsMaster', 'SimilarTo', 'IsSpliced', 'SpliceHt', 'Color', 'GUID'], axis=1)

# Rename height column to Story Height (m)
storyDefinitions_df = storyDefinitions_df.rename(columns={'Height': 'Story Height (m)'})

# Convert all data in story height column to metre from millimeter.
storyDefinitions_df['Story Height (m)'] = storyDefinitions_df['Story Height (m)'].astype(int)
storyDefinitions_df['Story Height (m)'] = storyDefinitions_df['Story Height (m)'] / 1000

# Insert Elevation (m) column.
storyDefinitions_df.insert(loc = 1, column= 'Elevation (m)', value = 0)

# Subtract the groundfloor story height by the basement height and add it to GF elevation. The basement height value must be altered manually.
storyDefinitions_df.at[storyDefinitions_df.index[-1], 'Elevation (m)'] = storyDefinitions_df.at[storyDefinitions_df.index[-1], 'Story Height (m)'] - 0.7 # CHANGE THIS VALUE 

# Loop through the dataframe and sum the story heights to get the total building elevation
for i in reversed(range(1, len(storyDefinitions_df))):
    storyDefinitions_df.loc[i - 1, 'Elevation (m)'] = storyDefinitions_df.loc[i, 'Elevation (m)'] + storyDefinitions_df.loc[i - 1, 'Story Height (m)'] # type: ignore


storyDefinitions_df.head()

,Story,Elevation (m),Story Height (m)
0,Top of Roof,179.250,7.695
1,Roof,171.555,3.630
2,L45,167.925,3.500
3,L44,164.425,3.500
4,L43,160.925,3.500


# <center>**Overall Wind Displacement in X & Y Direction - Maximum & Minimum**<center>

In [209]:
# Please manually select the load combination you wish to assess. It will automatically be loaded in.

# Grab the diaphragm center of mass displacements table.
mass_displacement_table = SapModel.DatabaseTables.GetTableForDisplayArray("Diaphragm Center Of Mass Displacements", GroupName = "") # type: ignore

# Extract the column headings and number of rows from the table. This needs to be done as the data is currently an array.
cols = mass_displacement_table[2]
num_rows = mass_displacement_table[3]

# Input data into array split.
table_data = np.array_split(mass_displacement_table[4], num_rows)

# Create wind displacement dataframe.
windDisplacement_df = pd.DataFrame(table_data)
windDisplacement_df.columns = cols

# Loop through the wind displacement dataframe and drop all rows containing non D1 diaphragms.
windDisplacement_df = windDisplacement_df[windDisplacement_df['Diaphragm'] == 'D1']

# Loop through the wind displacement dataframe and drop all rows containing min. This will be the max dataframe.
max_windDisplacement_df = windDisplacement_df[windDisplacement_df['StepType'] == 'Max']

# Loop through the wind displacement dataframe and drop all rows containing Max. This will be the min dataframe.
min_windDisplacement_df = windDisplacement_df[windDisplacement_df['StepType'] == 'Min']

max_windDisplacement_df.head()


,Story,Diaphragm,OutputCase,CaseType,StepType,StepNumber,StepLabel,UX,UY,RZ,Point,X,Y,Z
0,Top of Roof,D1,WINDDRIFT-CODE,Combination,Max,None,None,158.352,221.371,0.001159,1375,-115.57,10961.43,179250
6,Roof,D1,WINDDRIFT-CODE,Combination,Max,None,None,150.051,211.491,0.001127,34575,-1565.93,10731.21,171555
8,L45,D1,WINDDRIFT-CODE,Combination,Max,None,None,146.246,205.977,0.00111,35045,-513.65,10762.69,167925
10,L44,D1,WINDDRIFT-CODE,Combination,Max,None,None,142.493,201.066,0.001092,35359,-510.45,10762.12,164425
12,L43,D1,WINDDRIFT-CODE,Combination,Max,None,None,138.757,196.071,0.001075,35405,-510.45,10762.12,160925
